In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import re
import string
from string import punctuation
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words('english'))
punctuation = punctuation + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [3]:
df_train = pd.read_csv("./sample_data/drugsComTrain_raw.csv", engine="python")
df_test = pd.read_csv("./sample_data/drugsComTest_raw.csv", engine="python") 

print ("The shape of the train set given is : ", df_train.shape)
print ("The shape of the test set given is : ", df_test.shape)

df_train.head()

The shape of the train set given is :  (161297, 7)
The shape of the test set given is :  (53766, 7)


,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [4]:
merge = [df_train, df_test]
df_data = pd.concat(merge)

print (df_data.shape)

df_data.head(10)

(215063, 7)


,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10,14-Mar-15,32
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1,9-Aug-16,11
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1


In [5]:
total_na = df_data.isnull().sum(axis = 0)['condition']
df_data = df_data.dropna(how='any', axis=0)

In [6]:
df_data.columns = df_data.columns.str.lower()

In [7]:
df_data.sort_values(['uniqueid'], ascending = True, inplace = True)
df_data.reset_index(drop = True, inplace = True)
df_data.head(10)


,uniqueid,drugname,condition,review,rating,date,usefulcount
0,0,Medroxyprogesterone,Abnormal Uterine Bleeding,"""Been on the depo injection since January 2015...",3,28-Oct-15,4
1,2,Medroxyprogesterone,Amenorrhea,"""I&#039;m 21 years old and recently found out ...",10,27-Oct-15,11
2,3,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I have been on the shot 11 years and until a ...",8,27-Oct-15,7
3,4,Medroxyprogesterone,Birth Control,"""Ive had four shots at this point. I was on bi...",9,26-Oct-15,12
4,5,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I had a total of 3 shots. I got my first one ...",1,25-Oct-15,4
5,6,Medroxyprogesterone,Birth Control,"""This is a horrible shot!! I got my first shot...",1,24-Oct-15,3
6,7,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I&#039;m 18 and got this for heavy bleeding. ...",5,22-Oct-15,6
7,8,Medroxyprogesterone,Abnormal Uterine Bleeding,"""Im 19 and have been having heavy and painful ...",5,21-Oct-15,2
8,9,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I&#039;m a 30 year old woman. I got the shot ...",2,21-Oct-15,5
9,10,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I&#039;m 17 years old and I got shot in Augus...",7,20-Oct-15,2


In [8]:
df_data.loc[df_data['rating'] == 1, :]['drugname'].value_counts().head(10)

Miconazole                            767
Ethinyl estradiol / norethindrone     728
Etonogestrel                          707
Levonorgestrel                        511
Nexplanon                             504
Ethinyl estradiol / norgestimate      464
Medroxyprogesterone                   443
Depo-Provera                          417
Tioconazole                           410
Ethinyl estradiol / levonorgestrel    385
Name: drugname, dtype: int64

In [9]:
df_data['date'] = pd.to_datetime(df_data['date'])

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
df_ = df_data[['condition', 'drugname', 'rating', 'review']]
df_['review'] = df_data['review'].str.replace("&#039;", "")
df_['review'] = df_['review'].str.replace(r'[^\w\d\s]',' ')

# Removing the stop words before plotting
stop_words = set(stopwords.words('english'))
df_['review'] = df_['review'].str.lower()
df_['review_1'] = df_['review'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))
df_review = " ".join(df_['review_1'])
tokenize = word_tokenize(df_review)

In [12]:
df_['condition']=df_['condition'].str.lower()
df_['drugname']=df_['drugname'].str.lower()
df_['drugname'] = df_['drugname'].str.replace(' ','_')
df_['condition'] = df_['condition'].str.replace(' ','_')


In [13]:
df_.head()

,condition,drugname,rating,review,review_1
0,abnormal_uterine_bleeding,medroxyprogesterone,3,been on the depo injection since january 2015...,depo injection since january 2015 bleeding sto...
1,amenorrhea,medroxyprogesterone,10,im 21 years old and recently found out i migh...,im 21 years old recently found might pcos have...
2,abnormal_uterine_bleeding,medroxyprogesterone,8,i have been on the shot 11 years and until a ...,shot 11 years month ago never 1 period even sp...
3,birth_control,medroxyprogesterone,9,ive had four shots at this point i was on bi...,ive four shots point birth control pills years...
4,abnormal_uterine_bleeding,medroxyprogesterone,1,i had a total of 3 shots i got my first one ...,total 3 shots got first one leaving hospital g...


In [14]:
df_['review_2']=df_['condition']+' '+ df_['drugname']+' '+df_['review_1']
print(df_['review_2'][:3])

0    abnormal_uterine_bleeding medroxyprogesterone ...
1    amenorrhea medroxyprogesterone im 21 years old...
2    abnormal_uterine_bleeding medroxyprogesterone ...
Name: review_2, dtype: object


In [15]:
result=[word_tokenize(sentence) for sentence in df_['review_1']]

In [16]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size = 100, window = 5, min_count=5, workers=4, sg=0)

In [17]:
model_result = model.wv.most_similar("depression")
print(model_result)

[('ocd', 0.7576772570610046), ('depressive', 0.6968246102333069), ('mdd', 0.6763017773628235), ('paranoia', 0.6256870031356812), ('ptsd', 0.6200547218322754), ('pmdd', 0.618139386177063), ('psychosis', 0.617372989654541), ('sadness', 0.5918662548065186), ('depressions', 0.5840392112731934), ('gad', 0.5828171968460083)]


In [18]:
model.wv.similarity('depression','methylfolate')

0.007875127

In [19]:
result2 = [word_tokenize(sentence) for sentence in df_['review_2']]

In [20]:
print(result2[:3])

[['abnormal_uterine_bleeding', 'medroxyprogesterone', 'depo', 'injection', 'since', 'january', '2015', 'bleeding', 'stopped', 'another', 'injection', 'june', 'didnt', 'bleed', 'month', 'started', 'spotting', 'mid', 'july', 'since', 'approx', 'august', '1st', 'bleeding', 'constantly', 'heavy', 'enough', 'use', 'pads', 'almost', 'november', 'im', 'sick', 'bleeding', 'forking', 'money', 'sanitary', 'pads', 'shouldnt', 'bleeding', 'well', 'long', 'currently', 'awaiting', 'ultrasound', 'see', 'happening', 'would', 'like', 'change', 'contraceptive', 'doctor', 'wont', 'let'], ['amenorrhea', 'medroxyprogesterone', 'im', '21', 'years', 'old', 'recently', 'found', 'might', 'pcos', 'havent', 'gotten', 'period', 'since', '2010', 'october', '13th', '2015', 'started', 'taking', 'medroxyprogesterone', '10mg', '10', 'days', 'actually', 'worked', 'period', 'came', '4', 'days', 'finishing', 'pill', 'started', 'light', 'cramping', 'light', 'bleeding', 'second', 'day', 'period', 'looking', 'forward', 'see

In [21]:
model2 = Word2Vec(sentences=result2, size = 100, window = 10, min_count=3, workers=4, sg=0)

In [32]:
model2.wv.similarity('guanfacine','depression')

0.083923675

In [23]:
model2_result = model.wv.most_similar("adhd")
print(model2_result)

[('add', 0.7182839512825012), ('narcolepsy', 0.7127938270568848), ('concerta', 0.7003629207611084), ('ritalin', 0.6992884874343872), ('vyvanse', 0.699020266532898), ('bipolar', 0.6986600160598755), ('autism', 0.6894449591636658), ('adderall', 0.6644290685653687), ('schizophrenia', 0.6643838882446289), ('ocd', 0.661220371723175)]


In [24]:
rate_ls = []

for i in df_data.drugname.unique():
    
    # Only consider drugs that have at least 10 ratings
    if np.sum(df_data.drugname == i) >= 10:
        rate_ls.append((i, np.sum(df_data[df_data.drugname == i].rating) / np.sum(df_data.drugname == i)))
    
avg_rate = pd.DataFrame(rate_ls)

avg_rate = avg_rate.sort_values(by=[1], ascending=False).reset_index(drop=True)
avg_rate[:10]

,0,1
0,Privine,10.000000
1,Zutripro,10.000000
2,Chlorpheniramine / hydrocodone / pseudoephedrine,10.000000
3,Drixoral Cold and Allergy,9.956522
4,Dexbrompheniramine / pseudoephedrine,9.941176
5,Emend,9.916667
6,Aprepitant,9.916667
7,Chlorpheniramine / phenylephrine,9.900000
8,Silver sulfadiazine,9.808511
9,Tegaserod,9.772727


In [25]:
df_['review_1'][:5]

0    depo injection since january 2015 bleeding sto...
1    im 21 years old recently found might pcos have...
2    shot 11 years month ago never 1 period even sp...
3    ive four shots point birth control pills years...
4    total 3 shots got first one leaving hospital g...
Name: review_1, dtype: object

In [26]:
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = model[word]
                else:
                    doc2vec = doc2vec + model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    return document_embedding_list

In [28]:
document_embedding_list = vectors(df_['review_1'])
print('문서 벡터의 수 :',len(document_embedding_list))

문서 벡터의 수 : 213855


In [29]:
# cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
# print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

In [30]:
'''
def recommendations(title):
    books = df_[['title', 'image_link']]

    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(df.index, index = df['title']).drop_duplicates()    
    idx = indices[title]

    # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = books.iloc[book_indices].reset_index(drop=True)

    fig = plt.figure(figsize=(20, 30))

    # 데이터프레임으로부터 순차적으로 이미지를 출력
    for index, row in recommend.iterrows():
        response = requests.get(row['image_link'])
        img = Image.open(BytesIO(response.content))
        fig.add_subplot(1, 5, index + 1)
        plt.imshow(img)
        plt.title(row['title'])
'''

"\ndef recommendations(title):\n    books = df_[['title', 'image_link']]\n\n    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.\n    indices = pd.Series(df.index, index = df['title']).drop_duplicates()    \n    idx = indices[title]\n\n    # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정.\n    sim_scores = list(enumerate(cosine_similarities[idx]))\n    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)\n    sim_scores = sim_scores[1:6]\n\n    # 가장 유사한 책 5권의 인덱스\n    book_indices = [i[0] for i in sim_scores]\n\n    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.\n    recommend = books.iloc[book_indices].reset_index(drop=True)\n\n    fig = plt.figure(figsize=(20, 30))\n\n    # 데이터프레임으로부터 순차적으로 이미지를 출력\n    for index, row in recommend.iterrows():\n        response = requests.get(row['image_link'])\n        img = Image.open(BytesIO(response.content))\n        fig.add_subplot(1, 5, index + 1)\n        plt.imshow(img)\n        plt.title(row['title'])\n"